In [1]:
import arcpy
from sys import argv

In [2]:
#Set workspace and variables 
arcpy.env.workspace=r"C:\Users\MarySeidell\Documents\ArcGIS\Projects\Pendleton_OR\Pendleton_OR_Routes.gdb"

PendletonOR_Suitability_Experiment_3m_Buffer = "PendletonOR_Suitability_Experiment_3m_Buffer" #Surface Location


String = "Pendleton_OR_" #used for file naming 
Value = time.strftime("%d%m%Y%H%M%S") #used for file naming
Simplification_Tolerance = "300 Meters"

In [3]:
#Convert polygon surface to Raster Surface
Raster_output = String+"_toRaster_"+Value
ToRaster = arcpy.conversion.PolygonToRaster("PendletonOR_Suitability_Experiment_3m_Buffer", value_field="score_v2", out_rasterdataset=Raster_output, cell_assignment="CELL_CENTER", priority_field="NONE", cellsize="0.0004", build_rat="BUILD")

In [4]:
#Projet Raster 
Project = String+"_toRaster_Project_"+Value
Raster_output = arcpy.management.ProjectRaster(in_raster= Raster_output, out_raster=Project, out_coor_system="PROJCS[\"US_National_Atlas_Equal_Area\",GEOGCS[\"GCS_Sphere_Clarke_1866_Authalic\",DATUM[\"D_Sphere_Clarke_1866_Authalic\",SPHEROID[\"Sphere_Clarke_1866_Authalic\",6370997.0,0.0]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-100.0],PARAMETER[\"Latitude_Of_Origin\",45.0],UNIT[\"Meter\",1.0]]", resampling_type="NEAREST", cell_size="45.6326806908288 45.6326806908286", geographic_transform=[], Registration_Point="", in_coor_system="", vertical="NO_VERTICAL")
Raster_Project = arcpy.Raster(Project)

In [5]:
#Project Origin and Destination Points
Origin = "TribalCenter"
Destination = "Pendleton_Lab"
Origin_Project = String+"Origin_Project_"+Value
Destination_Project = String+"Dest_Project_"+Value
Origin_output = arcpy.management.Project(in_dataset=Origin, out_dataset=Origin_Project, out_coor_system="PROJCS[\"US_National_Atlas_Equal_Area\",GEOGCS[\"GCS_Sphere_Clarke_1866_Authalic\",DATUM[\"D_Sphere_Clarke_1866_Authalic\",SPHEROID[\"Sphere_Clarke_1866_Authalic\",6370997.0,0.0]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-100.0],PARAMETER[\"Latitude_Of_Origin\",45.0],UNIT[\"Meter\",1.0]]", transform_method=[], in_coor_system="GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]", preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")
Destination_output = arcpy.management.Project(in_dataset=Destination, out_dataset=Destination_Project, out_coor_system="PROJCS[\"US_National_Atlas_Equal_Area\",GEOGCS[\"GCS_Sphere_Clarke_1866_Authalic\",DATUM[\"D_Sphere_Clarke_1866_Authalic\",SPHEROID[\"Sphere_Clarke_1866_Authalic\",6370997.0,0.0]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-100.0],PARAMETER[\"Latitude_Of_Origin\",45.0],UNIT[\"Meter\",1.0]]", transform_method=[], in_coor_system="GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]", preserve_shape="NO_PRESERVE_SHAPE", max_deviation="", vertical="NO_VERTICAL")

In [6]:
#Run Least Cost Path Tool
LeastCostPath = String+"LeastCostPath_"+Value 
Path_output, Start_output, End_output = arcpy.intelligence.LeastCostPath(in_cost_surface=Raster_Project, in_start_point=Origin_Project, in_end_point=Destination_Project, out_path_feature_class=LeastCostPath, handle_zeros="SMALL_POSITIVE")

In [7]:
#Create Barrier
Barrier, Count = arcpy.management.SelectLayerByAttribute(in_layer_or_view=PendletonOR_Suitability_Experiment_3m_Buffer, selection_type="NEW_SELECTION", where_clause="score_v2 > 4", invert_where_clause="")

In [8]:
#Simplify Line
Simplify_Line = String+"SimplifyLine_"+Value
Simplify_Line_Output = arcpy.cartography.SimplifyLine(in_features=Path_output, out_feature_class=Simplify_Line, algorithm="WEIGHTED_AREA", tolerance=Simplification_Tolerance, error_resolving_option="RESOLVE_ERRORS", collapsed_point_option="NO_KEEP", error_checking_option="CHECK", in_barriers=[Barrier])[0]

In [9]:
#Generalize Line Further 
Generalize_Line_Output = arcpy.edit.Generalize(in_features=Simplify_Line_Output, tolerance="100 Meters")[0]

In [14]:
out_json = String+"_toGeoJSON_"+Value
GeoJson_Output = arcpy.FeaturesToJSON_conversion(in_features =Generalize_Line_Output, out_json_file = out_json, format_json="FORMATTED", geoJSON="GEOJSON")